In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [3]:

# Load the data from CSV
data = pd.read_csv('Occupancy.csv', usecols=[1, 2], nrows=100)


In [4]:

# Split the data into training and testing sets
train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)


In [11]:

# Define the model architecture
model = Sequential([
    Dense(64, activation='relu', input_shape=[1]),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.RMSprop(0.001), metrics=['mae', 'mse'])

# Train the model
history = model.fit(train_data['Humidity'], train_data['Temperature'], epochs=100, validation_split=0.2, verbose=0)

# Predict the temperature based on humidity
new_data = np.array([[40], [55]])
predictions = model.predict(new_data)

# Print the predictions
print(predictions)


1/1 [==============================] - 0s 59ms/step
[[33.510506]
 [45.92587 ]]


In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("converted_model.tflite2", "wb").write(tflite_quant_model)

INFO:tensorflow:Assets written to: C:\Users\JIRKA~1.DES\AppData\Local\Temp\tmpuxe4mmk4\assets


INFO:tensorflow:Assets written to: C:\Users\JIRKA~1.DES\AppData\Local\Temp\tmpuxe4mmk4\assets
c:\toi\tensorflow-lite-esp32-master\venv\Lib\site-packages\tensorflow\lite\python\convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


7408